In [128]:
# 環境變數
PORT = 3306
HOST = "10.238.0.7"
USER = "root"
PASSWORD = "bdse28001"
DATABASE = "bdse28stock"
TABLE = "news"

In [129]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.chrome.options import Options
# from selenium.common.exceptions import TimeoutException
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# import undetected_chromedriver as uc
from time import sleep
# import json
from datetime import datetime
from datetime import timedelta
import re
from pprint import pprint
import pymysql

In [130]:
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")  # 最大化視窗
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")  # 取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  # 設定為正體中文
my_options.add_argument('blink-settings=imagesEnabled=false')  # 不載入圖
my_options.add_experimental_option(
    "excludeSwitches", ['enable-automation', 'enable-logging'])  # 沒有異常log
driver = webdriver.Chrome(
    options=my_options,
    service=Service(ChromeDriverManager().install()))

In [131]:
# # schema
# db = pymysql.connect(host=HOST, port=PORT, user=USER, 
#                         password=PASSWORD, db=DATABASE)

# cursor = db.cursor()

# cursor.execute("DROP TABLE IF EXISTS news")
# sql = """CREATE TABLE `news` (
#   `news` char(20) NOT NULL,
#   `link` char(200) PRIMARY KEY NOT NULL,
#   `title` char(200) NOT NULL,
#   `time` datetime,
#   `reporter` char(200),
#   `article` varchar(3000) NOT NULL,
#   `stocks` varchar(50)
# ) ENGINE=InnoDB DEFAULT CHARSET=utf8;"""
# cursor.execute(sql)

# db.close()

In [132]:
url = 'https://ctee.com.tw/category/news/stocks'
driver.get(url)

In [133]:
alinks = []
results = []

In [134]:
def inputlink():
    links = driver.find_elements(
        By.CSS_SELECTOR, 
        "#content > div > div > div.col-sm-8.content-column > div.listing.listing-blog.listing-blog-5.clearfix > article > div > h2 > a"
    )
    times = driver.find_elements(
        By.CSS_SELECTOR, 
        "#content > div > div > div > div > article > div > div> span > time"
    )

    ctime = datetime.now()
    if ctime.hour == 7: # 7:30
        # yesterday 21:00 - today 7:30
        start = ctime.replace(hour=21, minute=0, second=0, microsecond=0) - timedelta(days=1)
        end = ctime.replace(hour=7, minute=30, second=0, microsecond=0)
    elif ctime.hour == 21: # 21:00
        # today 7:30 - today 21:00
        start = ctime.replace(hour=3, minute=30, second=0, microsecond=0)
        end = ctime.replace(hour=21, minute=0, second=0, microsecond=0)
    
    start = ctime.replace(hour=7, minute=30, second=0, microsecond=0)
    end = ctime.replace(hour=21, minute=0, second=0, microsecond=0)
    for i in range(len(links)):
        article_datetime = times[i].get_attribute('datetime')
        article_date_str = re.split('\+', article_datetime)[0][2:]
        article_date_time = datetime.strptime(article_date_str, '%y-%m-%dT%H:%M:%S')
        if article_date_time > end:
            continue
        elif article_date_time >= start:
            href = links[i].get_attribute('href')
            alinks.append(href)
        else:
            return False
    return True

In [135]:
def craw():
    title = driver.find_element(
            By.CSS_SELECTOR, 'div.post-header.post-tp-1-header > h1 > span').get_attribute("innerText")
    article_datetime = driver.find_element(
            By.CSS_SELECTOR, 'div.post-header.post-tp-1-header > div.post-meta-wrap.clearfix > div > span > time').get_attribute('datetime')
    article_date_str = re.split('\+', article_datetime)[0].replace('T', ' ')

    article_list = driver.find_elements(
            By.CSS_SELECTOR, 'div.entry-content.clearfix.single-post-content > p')
    article = ''
    for a in article_list:
        article += a.get_attribute("innerText")

    reporter_l = article.split('延伸閱讀')[0] # exists recommend
    reporter = reporter_l.split()[-1][:-1]
    if reporter == '': # exists space after name
        reporter = reporter_l.split()[-2] # get the before one
    if len(reporter) > 3: # no name
        try:
            reporter = driver.find_element(
                    By.CSS_SELECTOR, '#content > div > div > div.col-sm-8.content-column > div.single-container > section > div.author-title.heading-typo > a > span').get_attribute("innerText")
            reporter = reporter.split()[-1]
        except:
            reporter = ''
    if reporter == '數位編輯':
        reporter = ''

    result = {
        'news': '工商時報',
        'link': driver.current_url, 
        'title': title, 
        'time': article_date_str, 
        'reporter': reporter, 
        'article': article
        }
    
    # INSERT into TABLE
    try:
        sql = 'INSERT INTO %s (news, link, title, time, reporter, article) VALUES ("%s", "%s", "%s", "%s", "%s", "%s");'\
                            % (TABLE, result['news'], result['link'], result['title'], result['time'], result['reporter'], result['article'])
        cursor.execute(sql)
        db.commit()
    except Exception as e:
        print(e)
        pass

In [136]:
if __name__ == '__main__':

    db = pymysql.connect(host=HOST, port=PORT, user=USER, 
                        password=PASSWORD, db=DATABASE)
    cursor = db.cursor()

    page = 2
    while(inputlink()):
        sleep(1)
        driver.get(url+'/page/'+str(page))
        page += 1
    for link in alinks:
        driver.get(link)
        craw()
        sleep(2)

    driver.quit()
    
    # save as json
    # filename_innerText = 'test_'
    # with open(f'{filename_innerText}'+'.json', "w", encoding='utf-8') as file:
    #     file.write(json.dumps(
    #         results, ensure_ascii=False, indent=4))
    
    db.close()